In [1]:
import os
import pandas as pd
import numpy as np
import faiss

In [ ]:
DATA_DIR = "data_full"

os.listdir(DATA_DIR)

## Metadata - Movie Information

In [ ]:
meta = pd.read_parquet(f"{DATA_DIR}/metadata.parquet")

In [ ]:
print(f"meta shape {meta.shape}")
meta.columns.tolist()

In [ ]:
meta.head(3)

In [ ]:
meta.info(memory_usage="deep")

In [ ]:
meta.isnull().sum().sort_values(ascending=False).head(18)

In [ ]:
meta.describe(include="all")

## Embeddings - Stores Semantic meanning

In [ ]:
emb = np.load(f"{DATA_DIR}/embeddings.npy")

In [ ]:
emb.shape

In [ ]:
emb.dtype

In [ ]:
emb.nbytes / (1024**3)

In [ ]:
emb[0][:10]

In [ ]:
np.linalg.norm(emb[0])

## FAISS index - fast search
This file is built FROM embeddings.
faiss.swigfaiss_avx2.IndexFlatIP type of faiss

In [ ]:
index = faiss.read_index(f"{DATA_DIR}/faiss_index.faiss")

In [ ]:
type(index)

In [ ]:
index.ntotal

In [ ]:
index.d

In [ ]:
movie_idx = 100

D, I = index.search(emb[movie_idx].reshape(1, -1), 5)

meta.iloc[I[0]][["title"]]

## development dataset creation

In [ ]:
DATA_FULL = "data_full"
DATA_DEV = "data_dev"

os.makedirs(DATA_DEV, exist_ok=True)

meta = pd.read_parquet(f"{DATA_FULL}/metadata.parquet")
emb = np.load(f"{DATA_FULL}/embeddings.npy").astype("float32")